# Separate FloSup data into smaller sections using Arcpy
## Write to Csv, return number of photos per section
### Jin-Si R. Over - jover@usgs.gov
### Last updated 7/23/2020

In [57]:
#ArcDesktop - 10.7 running python 2.7
#Note, should run in python 3...

import os
import sys
try:
    import archook #The module which locates arcgis
    archook.get_arcpy()
    import arcpy
except ImportError:
    print ("can't find arcpy")
    
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from scipy import interpolate, signal
import glob
import pandas as pd
import csv
import copy

from arcpy import env
from arcpy.sa import *
import arcpy.cartography as CA

arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")
env.overwriteOutput = True
arcpy.overwriteOutput = True

In [62]:
path = r"C:/Users/jover/Working/02_USGS/01b_OBX_Florence/"
export_path = r"C:/Users/jover/Working/02_USGS/01b_OBX_Florence/001_FlightZips"

print("Setting workspace to the following directory: " + path)

wkspc				= path 

# Set paths and names
locgdb 				= wkspc + 'Florence_Photo_Locations.gdb/'		# Geodatabase location
locshp				= wkspc + '02_Shapefiles/'			# Shapefiles
loccsv              = wkspc + '001_FlightZips/' #Place the names for each section
env.workspace       = locgdb

Setting workspace to the following directory: C:/Users/jover/Working/02_USGS/01b_OBX_Florence/


In [87]:
#Set up floating ranges to call in a loop
step = 0.05                                        #Change to set the range of the latitude. 0.05 decimal degrees is 3 minutes
first = np.arange(33.8, 36.2, step)                #Lower bounding range of latitude floats that I want to separate out
second = np.arange(33.8 + step, 36.2 + step, step) #Upper bounding range based on step
zips = range(0,len(first),1)                       #Just an index for naming the output csvs


49

In [88]:
#Loop to select out file names from Florence by .05 (or whatever step specifed) decimal degrees

for i in range(len(first)):
    #Make a working layer to access from a point shapefile containing all the locations/data
    #The shapefile is the 3 merged Flight Locations csv files from Wayne from 2018
    arcpy.MakeFeatureLayer_management(locshp + "Florence_PhotoLoc.shp",'Florence_PhotoLoc_lyr')
    
    #Select from the layer by the latitude attribute and the ranges specified in the cell above
    sectionPhotos = arcpy.SelectLayerByAttribute_management('Florence_PhotoLoc_lyr',"NEW_SELECTION","Latitude > " + str(first[i]) + " AND Latitude < " + str(second[i]))
    
    #Creates copy of the selected features and puts them into a gdb feature class that is overwritten each time
    arcpy.CopyFeatures_management(sectionPhotos, locgdb + "overwrite")
    
    #Takes the file being overwritten and makes it into a csv with an incremental name
    arcpy.TableToTable_conversion(locgdb + "overwrite",'C:/Users/jover/Working/02_USGS/01b_OBX_Florence/001_FlightZips/','Zip_' + str(zips[i]) + '.csv','',field_mapping = )


In [91]:
#Get rid of extraneous columns in the csv created by Arc

#Walk through the created csv, read them using pandas and then rewrite it only using the columns we want to keep
for f in glob.glob("C:/Users/jover/Working/02_USGS/01b_OBX_Florence/001_FlightZips/*.csv"):
    fn = pd.read_csv(f)
    keep_col = ['Station','UTC_Time', 'Latitude', 'Longitude', 'Easting','Northing']
    new_f = fn[keep_col]
    new_f.to_csv(f)

In [59]:
#See how many photos are in each file and write out as a csv - https://stackoverflow.com/questions/31616217/python-script-to-count-num-lines-in-all-files-in-directory

os.chdir('C:/Users/jover/Working/02_USGS/01b_OBX_Florence/01_Flight_Zip_Sections/')
names={}
for fn in glob.glob('*.csv'):                          #Walk through every csv in the zipped sections
    with open(fn) as f:
        names[fn]=sum(1 for line in f if line.strip()) #counts the lines and sums them for each csv
    
with open('Photo_count.csv','wb') as f:                #Save as a csv
    w = csv.DictWriter(f,names.keys());
    w.writeheader();
    w.writerow(names)                                  #First row is the names of the csvs, second is the count
    
